# DESCRIÇÃO DO TRABALHO

Nestre trabalho, vou procurar apresentar de forma visualmente adequada a distribuição de porcentagens de confianças das predições das classes na detecção de objetos com YOLO. É de interesse do pesquisador de detecção de objetos, além de saber se o modelo usado acertou ou não na predição da classe do objeto, saber os indices de confianças para as demais classes, pois em alguns casos, apesar do modelo ter acertado a classe do objeto, ele pode ter quase errado ou confundido.


Por motivos de limitações de recursos (Não possuir GPU para treinar um modelo) e tempo, usarei um modelo pré-treinado do Yolo feito com o Dataset COCO. Isso não afeta o trabalho, pois a visualização das predições não dependem de qual modelo está sendo utilizado. É utilizado 10 imagens das 10 classes de animais do COCO Dataset, que estão em diversos cenários. As classes são:

'bird','cat','dog','horse','sheep','cow','elephant','bear','zebra','giraffe'


Será disponibilizado no .zip da entrega:

Um arquivo __.ipynb__ gerado pelo COLAB com a execução dos códigos deste relatório
Uma pasta de anotações do COCO Dataset (mas pode ser baixada pelo arquivo do COLAB)
Uma pasta de arquivos de configuração da rede Yolo (.cfg e .names)
Arquivo __yolov4.weights__ que são os pesos da rede pré-treinada (mas pode ser baixada pelo arquivo do COLAB)
Um CSV com os dados finais
Este arquivo



# Download das imagens de validação

Para baixar as imagens, precisamos utilizar a pasta de anotações do COCO Dataset (Disponível na entrega) ou baixar e extrair pelo Colab com o comando:

``!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip``

``!unzip annotations_trainval2017.zip``


Agora você precisa instalar o pacote da api do COCO para realizar o download das imagens no Colab:

`` !pip install pycocotools  ``


Fazer os imports das dependências

``
import cv2
import numpy as np
import time
import os
from os import listdir
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import zipfile
from os.path import isfile, join
import csv 
from pycocotools.coco import COCO
import requests
``

e por fim, com as anotações e o pacote da api instalado, executar o código abaixo.

```
def createDirectory(name, animal):
    import os

 
    path = name
    access_rights = 0o755
    try:
        os.mkdir(path, access_rights)
    except OSError:
        print ("Creation of the directory %s failed" % path)
        
    else:
        print ("Successfully created the directory %s " % path)

    os.chdir(name)


    path = animal
    access_rights = 0o755
    try:
        os.mkdir(path, access_rights)
    except OSError:
        print ("Creation of the directory %s failed" % path)
        
    else:
        print ("Successfully created the directory %s " % path)

    os.chdir("..")
```

```
def downloadValid(animal, coco, index):
    createDirectory('valid/', animal)
    cats = coco.loadCats(coco.getCatIds())
    nms=[cat['name'] for cat in cats]

    catIds = coco.getCatIds(catNms=[animal])

    imgIds = coco.getImgIds(catIds=catIds )
    images = coco.loadImgs(imgIds)

    i = 1
    for im in images:
        if i <= 4:
            img_data = requests.get(im['coco_url']).content
            
            with open('valid/'+ animal + '/'+ animal+'_'+im['file_name'], 'wb') as handler:
                handler.write(img_data)
            
            time.sleep(3)    
            i = i + 1
        else:
            break
```

``
coco = COCO('annotations/instances_val2017.json')
animals = [ 'bird',
            'cat',
            'dog',
            'horse',
            'sheep',
            'cow',
            'elephant',
            'bear',
            'zebra',
            'giraffe']
for item in animals:
  downloadValid(item, coco, animals.index(item))
``


Irá ser criado no diretório raiz, uma pasta chamada __valid__ com 10 sub-pastas, referenciando cada classe e suas imagens.


# Passos para extrair as confianças das predições do modelo Yolo com OpenCV

Para a execução, utilizar o Google Colab sem GPU pois o modelo já está treinado. Lembrando que estou interessando em apresentar as confianças de cada predição feita, portanto, será mostrado o processo de predição mas não códigos de como apresentar as caixas delimitadoras (bounding-boxes). 

No ambiente, é necessário instalar o Open CV na versão 4.4, que suporta o Yolo.

__!pip install opencv-python==4.4.0.40__


Verificando a versão do open cv

``print(cv2.__version__)``


Baixe os pesos do modelo pré-treinado.

``!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights``

__yolov4.weights__ (São os pesos da rede neural convolucional)

Fazer o upload dos arquivos da pasta cfg (na entrega), contendo os arquivos (O Colab não irá deixar fazer o upload da pasta, portanto, cria uma pasta 'cfg' e dentro dela faça o upload dos dois arquivos dentro dela)

__yolov4.cfg__ (Arquivo de configuração da rede)
__coco.names__ (Contém os nomes de todas as classes treinadas, inclusive as classes que pretendo testar)

Guardando em uma lista, as classes contidas no arquivo __coco.names__ (lembrando que no google Colab, o diretório raiz de manipulação é __/content/__) 

Obs: existem 80 classes neste modelo pré-treinado, então haverá várias classes onde as futuras confianças assumirão valor 0%, pois irei somente utilizar as imagens de animais.

``
labelsPath = os.path.sep.join(['/content/cfg', 'coco.names'])
LABELS = open(labelsPath).read().strip().split('\n')
``

Guardando os pesos e as configurações da cnn em duas varias respectivamente

``
weightsPath = os.path.sep.join(['/content', 'yolov4.weights'])
configPath = os.path.sep.join(['/content/cfg', 'yolov4.cfg'])
``

Agora iremos criar a rede com o OpenCV e pegar somente as 3 camadas de saída

``
net = cv2.dnn.readNet(configPath, weightsPath)
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
``


Agora iremos criar duas funções, uma para criar um CSV final, com as predições e outra função para gerar as predições.

Para o trabalho, estes dados serão exportados para um __CSV__, que facilitará as próximas etapas do projeto da disciplina. Pois não dependerá de executar mais de uma vez códigos de download de imagens e a execução destes passos e sim, só se preocupar com os dados finais. Além disso, facilitará a exportação para usar no Jupyter.

Será armazenado em um __CSV__ os seguintes dados: Nome da imagem processada, a  camada de saída da rede neural e 80 colunas, que correspondem cada classe do modelo pré-treinado usado. Totalizando 82 COLUNAS. 

O parâmetro Threshold, define uma porcentagem mínima de confiança para filtrar as confianças baixas. No caso iremos definir como 0.4 (tudo acima de 40% de confiança será retornado). Este valor pode ser alterado para ver o quão o modelo consegue de precisão máxima sem perder todas as predições e depende do problema em questão.

O YoloV4 possui 3 camadas de saída, e elas retornam várias predições. Estas predições são formadas por arrays que possuem a confiança para cada classe detectada. Cada array possui várias posições, as 4 primeiras posições do array identificam as coordenadas na imagem para a formação da caixa delimitadora, a partir do indice 5, ficam as confianças para cada classe do modelo para aquela predição executada.


PRÉ-PROCESSAMENTO

É aplicado a função do OpenCV denominada __cv2.dnn.blobFromImage__. Esta função aplica dois pré-processamentos, o Mean subtraction e o Scaling.

A Mean subtraction é usada para ajudar a reduzir o impacto das mudanças de iluminação nas imagens de entrada. 
Primeiro calculamos a intensidade de iluminação média dos pixels em todas as imagens para cada um dos três canais (RGB).

Nesta etapa, logo após, aplicaremos o Scaling. Ele procura realizar uma normalização dos resutlados do Mean Subtraction de acordo com um fator de escala.  Este valor padrão é 1.0 (ou seja, sem mudança na escala), mas podemos fornecer outro valor também. Ele pode ser divisível pelo desvio padrão das dimensões das imagens do conjunto.

A função recebe como parâmetros:
a imagem, 
o fator de escala (1 / 255.0), 
uma tupla com as dimensões desejadas (Para o YoloV4 é interessante utilizar 416x416), 
se irá ser utilizado o Mean Subtraction (definindo swapRB como True),
se a imagem vai ser recortada ou não



(Fonte: https://www.pyimagesearch.com/2017/11/06/deep-learning-opencvs-blobfromimage-works/ e https://docs.opencv.org/3.4.15/d6/d0f/group__dnn.html)


```
rows = [] 
def createCSV(rows):
  
  Details = ['ImageName', 'LayerOutput']
  for item in LABELS:
    Details.append(item)
  
  with open('confiancasDasDeteccoes.csv', 'w') as f: 
    write = csv.writer(f) 
    write.writerow(Details) 
    write.writerows(rows)

def gerandoConfiancas(imagem, imgName ):

  threshold = 0.2
  confiances = []
  idClasses = []
  dados = []
  currentLayer = 0

  blob = cv2.dnn.blobFromImage(imagem, 1 / 255.0, (416,416), swapRB=True, crop=False)
  net.setInput(blob)
  layer_outputs = net.forward(ln)

  for output in layer_outputs:
    for detection in output:
      scores = detection[5:] #Filtrando somente as confianças
      classeID = np.argmax(scores)
      confiance = scores[classeID]
      dados = []
      if confiance > threshold: #aplicando a filtragem do Threshold
          dados = [imgName, currentLayer]
          for item in scores:
            dados.append(item)
          print('scores: '+ str(scores)) #Apresenta todas as confianças
          print('classe mais provável: '+ str(classeID)) #Apresenta o index da classe.
          print('confiança: '+ str(confiance))#Apresenta a confiança daquela classe que foi apontada como a mais confiante

          confiances.append(float(confiance))
          idClasses.append(classeID)
        
          rows.append(dados)
    
    currentLayer = currentLayer + 1
    
    
```

Executando as funções. Ao final, será gerado um CSV e poderá ser baixado, clicando nas opções do arquivo. 

```
animals = [ 'bird',
            'cat',
            'dog',
            'horse',
            'sheep',
            'cow',
            'elephant',
            'bear',
            'zebra',
            'giraffe']

for item in animals:

  mypath = f"/content/valid/{item}"
  onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
  for img in onlyfiles:
    imagem = cv2.imread(f'/content/valid/{item}/{img}')
    gerandoConfiancas(imagem, img)

```


